In [1]:
# import dependencies
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# initialize variables
min_date = pd.Timestamp('1900-01-01')
max_date = pd.Timestamp('2023-09-28')

In [3]:
# set paths 
folder_path = '../Data Set/Data_archive/'
exclude_files = ['NIFTY50_all.csv', 'stock_metadata.csv', 'INFRATEL.csv']
output_folder = '../Data Set/'

### PRE-PROCESS THE DATA

In [4]:
# determine the optimal data range to be kept 
for file in os.listdir(folder_path):
    if file in exclude_files:
        continue

    file_path = os.path.join(folder_path, file)
    stock_data = pd.read_csv(file_path)
    
    # for each stock file select its min and max date.
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    stock_data['Symbol'] = stock_data['Symbol'].iloc[0]
    start_date = stock_data['Date'].min()
    end_date = stock_data['Date'].max()

    # max of all oldest dates is taken as lower bound.
    if(start_date > min_date):
        min_date = start_date
    
    # min of all recent dates is taken as upper bound.
    if(end_date < max_date):
        max_date = end_date


In [5]:
max_date

Timestamp('2021-04-30 00:00:00')

In [6]:
min_date

Timestamp('2010-11-04 00:00:00')

In [7]:
with open(output_folder+"processedData.csv", 'w') as csv_file:
    # Write the header row
    header = 'Date,Symbol,Close'
    csv_file.write(header + '\n')

In [8]:
# Save the data
for file in os.listdir(folder_path):
    if file in exclude_files:
        continue

    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path)
    
    # in case of where the name of the stock has changes, we take entire data under its current name.
    data['Date'] = pd.to_datetime(data['Date'])
    stock_data = data[data['Date'] < max_date][data['Date'] >= min_date]   
    stock_data['Symbol'] = stock_data['Symbol'].iloc[-1]
    
    # Save only the required columns i.e the closing price of a stock for all dates.
    stock_data[['Date', 'Symbol','Close']].to_csv(output_folder+"processedData.csv", mode='a', header=False, index=False)


### SAVE IT IN A 2-D MATRIX FORMAT

In [9]:
# create a dummy 2-D table 
r = open(output_folder+"ClosingPrices.csv", 'w')

In [10]:
data = pd.read_csv(output_folder+"processedData.csv")
data['Date'] = pd.to_datetime(data['Date'])

In [11]:
# Find all stocks and dates, we posess data of.
x = data['Symbol'].unique()
dates = []
for i in data['Date'].unique():
    dates.append(str(i)[:10])
dates.sort()

In [12]:
# Add all column (stocks) and row (dates) name to the table
with r as csv_file:
    header = 'Date,'
    csv_file.write('Date,')
    for i in range(len(x)-1):
        csv_file.write(x[i] + ',')
    csv_file.write(x[-1])
    csv_file.write('\n')
    for i in dates:
        csv_file.write(i + '\n')

In [13]:
# Update the value from the dummy table with the actual closing prices.
closingPrices = pd.read_csv(output_folder+'ClosingPrices.csv', index_col=0)
for index, rows in data.iterrows():
    closingPrices[rows['Symbol']][str(rows['Date'])[:10]] = rows['Close']

In [14]:
# save the change
closingPrices.to_csv(output_folder+'ClosingPrices.csv') 